# HuggingFace Diffuser Library

- HuggingFace에서 제공하는 diffusion 모델을 위한 라이브러리
- Diffusion 모델을 사용하여 추론하는 것과 모델 학습을 지원한다.
- https://huggingface.co/docs/diffusers/v0.31.0/ko/index

## 주요 구성요소
- **DiffusionPipeline**
    - 추론을 위해 사전 학습된 diffusion 모델에서 샘플을 빠르게 생성하도록 설계된 높은 수준의 엔드투엔드 클래스.
- **사전학습된 Model 및 모듈**
    - Diffusion 모델 생성을 위한 layer block으로 사용할 수 있는 사전 학습된 model 아키텍처 및 모듈.
- **다양한 scheduler들**
    - 학습을 위해 노이즈를 추가하는 방법과 추론 중에 노이즈 제거된 이미지를 생성하는 방법을 제어하는 알고리즘.

### DiffusionPipeline
- 지원 task

|Task|	Description|	Pipeline|
|-|-|-|
|Unconditional Image Generation|Gaussian noise로 부터 이미지 생성|	unconditional_image_generation|
|Text-Guided Image Generation|text prompt에 맞는 이미지를 생성|	conditional_image_generation|
|Text-Guided Image-to-Image Translation|이미지와 text prompt가 주어지면 주어진 이미지를 text prompt에 맞춰 변경|	img2img|
|Text-Guided Image-Inpainting|주어진 이미지의 일부분을 text prompt에 맞춰 변경|	inpaint|
|Text-Guided Depth-to-Image Translation|이미지와 text prompt를 입력 받아 이미지의 구조는 유지하면서 text prompt에 맞춰 이미지를 변경|depth2img|

In [ ]:
%pip install -U diffusers accelerate transformers tqdm ipywidgets

In [ ]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
from diffusers import StableDiffusionPipeline
import torch

model_id = "dreamlike-art/dreamlike-photoreal-2.0"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to(device)

In [ ]:
prompt = "photo, a church in the middle of a field of crops, bright cinematic lighting, gopro, fisheye lens"
result_image = pipe(prompt)

In [ ]:
image_list = result_image.images
image = result_image.images[0]

image.save("./result_1.jpg")
image.show()

In [ ]:
prompt = "London streets in the 1800s. A scene with a tram running on the street and mercury lamps." 
images = pipe(
    prompt, 
    num_inference_steps=100,
    guideance_scale=10,
)

In [ ]:
images.images[0].save("result_2.jpg")
images.images[0]

In [ ]:
prompt_list = [
    "photo of cat running on the loop.",
    "japanese manga style illustration of boy and girl eatting ice cream.",
]
image_list2 = pipe(prompt_list, num_images_per_prompt=2)

In [ ]:
image_list2.images

In [ ]:
for idx, img in enumerate(image_list2.images, start=10):
    img.save(f"./result_{idx}.jpg")

#### torch.Generator 를 이용해 항상 동일한 이미지 생성
- torch.Generator
    -  PyTorch에서 난수 생성기를 관리하는 객체.
    -  난수 생성 함수에서 일관된 결과를 얻기 위해 사용된다.
-  메소드
    -  `manual_seed(seed)`: 난수 생성을 위한 시드를 설정. 시드값이 같으면 같은 난수를 생성한다.

In [ ]:
## Generator 예제
import torch

# CPU
g_cpu = torch.Generator()
g_cpu.manual_seed(1234)
rand_tensor = torch.rand((2,2), generator=g_cpu)
rand_tensor

# CUDA
# g_cuda = torch.Generator(device='cuda')
# g_cuda.manual_seed(5678)
# rand_tensor_cuda = torch.rand(3, generator=g_cuda, device='cuda')
# rand_tensor_cuda

In [ ]:
# 동일한 이미지 생성
generator = torch.Generator(device=device).manual_seed(100)
prompt = "London streets in the 1800s. A scene with a tram running on the street and mercury lamps." 
image = pipe(
    prompt,
    num_inference_steps=30,
    num_images_per_prompt=2, 
    generator=generator
)

In [ ]:
image.images[0]

In [ ]:
image.images[1]

# Stable Diffusion Image 생성 툴

- ComfyUI
    - https://github.com/comfyanonymous/ComfyUI
- Stable Diffusion web UI
    - https://github.com/AUTOMATIC1111/stable-diffusion-webui